<a href="https://colab.research.google.com/github/DragosBrex/portofolio/blob/main/Machine_Translation_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importam librariile necesare
import pandas as pd
import numpy as np
import pathlib
import random
import string
import re
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
import collections
import helper

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras import Sequential

In [ ]:
#Conectare Colab la Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Citirea dataset-ului din Google Drive
!cp '/content/drive/My Drive/Diacritice Model - Folder/data.csv' .

In [ ]:
#Afisarea fisierelor citite
!ls

data.csv  drive  sample_data


In [ ]:
#Citirea datelor din dataset intr-o variabila numita culmea "data"
data = pd.read_csv("data.csv")
"data"

'data'

In [ ]:
#Definim o variabila care sa contina semnele de punctuatie si caracterele speciale
strip_chars = string.punctuation 
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
strip_chars

'!"#$%&\'()*+,-./:;<=>?@\\^_`{|}~'

In [ ]:
#Simplificam datele, stergand semnele de punctuatie si caracterele speciale. Apoi transformam toate literele in litere mici

for line in range(len(data["Input"])):
  for caracter in data["Input"][line]:
    if strip_chars.find(caracter) >= 0:
      data["Input"][line] = data["Input"][line].replace(caracter, "")
      data["Output"][line] = data["Output"][line].replace(caracter, "")

for line in range(len(data["Input"])):
  data["Input"][line] = data["Input"][line].lower()
  data["Output"][line] = data["Output"][line].lower()

In [ ]:
#Amestecam datele si le impartim in date pentru test 
data = shuffle(data) 

train_size = int(len(data)*0.8)
test_size = int(len(data)*0.2)


train_data = data[:train_size]
test_data = data[train_size + 1 : ]

print("Train size: %d" % train_size)
print("Test size: %d" % test_size)

print("Total size: %d" % (train_size+test_size))


Train size: 5316
Test size: 1329
Total size: 6645


In [ ]:
#Definim o functie ce va transorma cuvintele noastre in id-uri numerice
def tokenize(x):
    
    tokenizer = Tokenizer(char_level=False) # char_level trebuie neaparat setat pe valoarea "False"
    tokenizer.fit_on_texts(x)

    return tokenizer.texts_to_sequences(x), tokenizer

In [ ]:
#Definim o functie ce va egala lungimea textelor ce vor fi comparate, deoarece... si aici explica Dragos
def pad(x, length=None):
    
    if length is None:
        
        length = max([len(sentence) for sentence in x])
        print(length)
        
    return pad_sequences(x, maxlen=length, padding='post')


In [ ]:
#Definim o functie care aplica cele doua functii de mai sus datelor noastre
def preprocess(x, y):

    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk


text_fd_train, text_cd_train, fd_romanian_tokenizer, cd_romanian_tokenizer = preprocess(train_data["Input"], train_data["Output"])
    
max_fd_length = text_fd_train.shape[1] # _fd_ = "fara diacritice"
max_cd_length = text_cd_train.shape[1]   # _cd_ = "cu diacritice"

fd_size = len(fd_romanian_tokenizer.word_index)
cd_size = len(cd_romanian_tokenizer.word_index)

print("Cea mai lunga secventa de text fara diacritice are %d" % max_fd_length + " de caractere")
print("Cea mai lunga secventa de text cu diacritice are %d" % max_cd_length + " de caractere")
print("Dimensiunea vocabularului:", cd_size)

21
21
Cea mai lunga secventa de text fara diacritice are 21 de caractere
Cea mai lunga secventa de text cu diacritice are 21 de caractere
Dimensiunea vocabularului: 10958


In [ ]:
#In final cream modelul si il antrenam
def the_model(input_shape, output_sequence_length, input_size, output_size):

    learning_rate = 1e-3 

    input_seq = Input(input_shape[1:])
    
    rnn = GRU(64, return_sequences=True)(input_seq)
    
    logits = TimeDistributed(Dense(output_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

fd_train_data = pad(text_fd_train, max_cd_length)
fd_train_data = fd_train_data.reshape(text_cd_train.shape)

cd_train_data = text_cd_train

#Antrenarea modelului
model = the_model(
    fd_train_data.shape,
    max_cd_length,
    fd_size,
    cd_size)

model.fit(fd_train_data, cd_train_data, batch_size=1024, epochs=50, validation_split=0.2)


Epoch 1/50
5/5 [==============================] - 4s 238ms/step - loss: 9.2422 - accuracy: 0.2868 - val_loss: nan - val_accuracy: 0.4314
Epoch 2/50
5/5 [==============================] - 1s 132ms/step - loss: 9.0676 - accuracy: 0.4247 - val_loss: nan - val_accuracy: 0.4313
Epoch 3/50
5/5 [==============================] - 1s 142ms/step - loss: 8.8385 - accuracy: 0.4247 - val_loss: nan - val_accuracy: 0.4313
Epoch 4/50
5/5 [==============================] - 1s 134ms/step - loss: 8.5092 - accuracy: 0.4247 - val_loss: nan - val_accuracy: 0.4313
Epoch 5/50
5/5 [==============================] - 1s 132ms/step - loss: 8.0285 - accuracy: 0.4247 - val_loss: nan - val_accuracy: 0.4313
Epoch 6/50
5/5 [==============================] - 1s 133ms/step - loss: 7.4751 - accuracy: 0.4246 - val_loss: nan - val_accuracy: 0.4310
Epoch 7/50
5/5 [==============================] - 1s 134ms/step - loss: 6.9843 - accuracy: 0.4245 - val_loss: nan - val_accuracy: 0.4310
Epoch 8/50
5/5 [=========================

In [ ]:
#Definim o functie ce transforma id-urile numerice inapoi in cuvinte
def logits_to_text(logits, tokenizer):

    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
text_fd_test, text_cd_test, fd_romanian_tokenizer, cd_romanian_tokenizer = preprocess(test_data["Input"], test_data["Output"])

max_fd_length = text_fd_test.shape[1] # _fd_ = "fara diacritice"
max_cd_length = text_cd_test.shape[1]   # _cd_ = "cu diacritice"

fd_size = len(fd_romanian_tokenizer.word_index)
cd_size = len(cd_romanian_tokenizer.word_index)

print("Cea mai lunga secventa de text fara diacritice are %d" % max_fd_length + " de caractere")
print("Cea mai lunga secventa de text cu diacritice are %d" % max_cd_length + " de caractere")
print("Dimensiunea vocabularului:", cd_size)

20
20
Cea mai lunga secventa de text fara diacritice are 20 de caractere
Cea mai lunga secventa de text cu diacritice are 20 de caractere
Dimensiunea vocabularului: 4580


In [ ]:
fd_test_data = pad(text_fd_test, max_cd_length)
fd_test_data = fd_train_data.reshape(text_cd_train.shape)

In [ ]:
cd_test_data = text_cd_test